In [1]:
!pip install dash jupyter-dash plotly pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [jupyter-dash] [dash]


In [6]:
# Import required libraries
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Assuming spacex_df is already loaded
# If not, you would need to load it here
# spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Create app layout
app.layout = html.Div(
    [
        html.H1(
            "SpaceX Launch Records Dashboard",
            style={"textAlign": "center", "color": "#503D36", "font-size": 40},
        ),
        
        # TASK 1: Add a dropdown list to enable Launch Site selection
        html.Div(
            [
                html.Div("Select a Launch Site:"),
                dcc.Dropdown(
                    id="site-dropdown",
                    options=[
                        {"label": "All Sites", "value": "ALL"},
                        {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
                        {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
                        {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                        {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"},
                    ],
                    value="ALL",
                    placeholder="Select a Launch Site",
                    searchable=True,
                ),
            ],
            style={"width": "80%"},
        ),
        
        html.Br(),
        
        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(dcc.Graph(id="success-pie-chart")),
        
        html.Br(),
        
        # TASK 3: Add a slider to select payload range
        html.Div(
            [
                html.Div("Payload range (Kg):"),
                dcc.RangeSlider(
                    id="payload-slider",
                    min=0,
                    max=10000,
                    step=1000,
                    marks={i: str(i) for i in range(0, 10001, 1000)},
                    value=[0, 10000],
                    # Fixed the syntax error here - removed the trailing comma
                    tooltip={"placement": "bottom", "always_visible": True}
                ),
            ],
            style={"width": "80%"},
        ),
        
        html.Br(),
        
        # TASK 4: Success vs Payload scatter plot
        dcc.Graph(id="success-payload-scatter-chart"),
    ],
    style={"margin": "0 auto", "width": "95%"},
)

# ---------- Callbacks ----------

# Pie chart: if ALL, show total success count by site; otherwise success vs failure for the selected site
@app.callback(Output("success-pie-chart", "figure"), Input("site-dropdown", "value"))
def update_pie(selected_site):
    if selected_site == "ALL":
        # Sum successes by site (class=1)
        agg = (spacex_df.groupby("Launch Site")["class"].sum().reset_index())
        fig = px.pie(
            agg, values="class", names="Launch Site",
            title="Total Successful Launches by Site"
        )
    else:
        filtered = spacex_df[spacex_df["Launch Site"] == selected_site].copy()
        # Map class 1/0 -> Success/Failure for clearer labels
        filtered["Outcome"] = filtered["class"].map({1: "Success", 0: "Failure"})
        fig = px.pie(
            filtered, names="Outcome",
            title=f"Success vs. Failure for {selected_site}"
        )
    return fig

# Scatter: filter by payload range and optionally by site
@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [Input("site-dropdown", "value"), Input("payload-slider", "value")],
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    mask = (spacex_df["Payload Mass (kg)"] >= low) & (spacex_df["Payload Mass (kg)"] <= high)
    df = spacex_df.loc[mask]
    if selected_site != "ALL":
        df = df[df["Launch Site"] == selected_site]
    fig = px.scatter(
        df,
        x="Payload Mass (kg)",
        y="class",
        color="Booster Version Category",
        title="Correlation between Payload and Launch Success",
        hover_data=["Launch Site"],
    )
    return fig

# Run in Jupyter notebook - using the updated method for JupyterDash
import jupyter_dash
from jupyter_dash import JupyterDash

# If using JupyterDash, replace the app initialization with:
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# For regular Dash in Jupyter, use this instead:
app.run(mode="jupyterlab", port=8050, debug=True)  